CONTAGION 
NODE BLOCKING
ERDOS-RENYI GRAPH


In [1]:
import matplotlib.pyplot as plt
from networkx import nx
import numpy as np
import algorithmx
import random
import pandas as pd
import os
import time
import gurobipy as gp
from gurobipy import GRB
import copy




list_n=[100]
list_m=[400]
# percentage listler büyükten küçüğe olmak zorunda!!!!
infected_percentage_list = [0.15,0.1,0.05]
risky_percentage_list = [0.6,0.5,0.4,0.3]
name = "atakan"


### node features klasörü açmayı unutma
### node features klasörü açmayı unutma
for elem in range(len(list_n)):
    random.seed(elem)
    n=list_n[elem]
    m=list_m[elem]
    G = nx.gnm_random_graph(n, m)
    colnames=range(1,(n+1))
    adjacency_df = pd.DataFrame(np.zeros((n,n)),columns=colnames,index=colnames)

    for (u, v) in G.edges():
        G.edges[u,v]['weight'] = np.random.randint(1,1000)/1000
        adjacency_df.iat[u,v]=G.edges[u,v]['weight']
        adjacency_df.iat[v,u]=G.edges[u,v]['weight']
    adjacency_df['Threshold']=np.random.randint(1,1000,size=n).astype("float")/1000 #Tekrar bak ...
    adjacency_df['IsInfected']=np.zeros((n,1))
    adjacency_df['IsRisky']=np.zeros((n,1))
    adjacency_df_original = adjacency_df.copy(deep=True)
    
    dummy_vector_for_selection=np.arange(1,n+1)
    max_infected = int(infected_percentage_list[0]*n)
    max_risky = int(risky_percentage_list[0]*n)
    pool_for_all = np.random.choice(dummy_vector_for_selection,(max_infected+max_risky),replace=False)
    pool_for_infected = np.random.choice(pool_for_all,max_infected,replace=False)
    pool_for_risky = [risk for risk in pool_for_all if risk not in pool_for_infected]
    for inf_rate in infected_percentage_list:
        for risky_rate in risky_percentage_list:
            
            infected_percentage=inf_rate
            total_risky_percentage=1-infected_percentage
            risky_percentage=risky_rate

            total_infected=int(infected_percentage*n)
            total_risky=int(risky_percentage*n)
            #nx.draw(G)
            #plt.show()
            selected_infected=pool_for_infected[:total_infected]
            selected_risky=pool_for_risky[:total_risky]
            

            adjacency_df=adjacency_df_original.copy(deep=True)
            adjacency_df.loc[selected_infected,'IsInfected']=1
            adjacency_df.loc[selected_risky,'IsRisky']=1

            os.chdir("C:\\Users\\Lenovo\\Desktop\\Experiments\\data\\adjaceny_df")
            
            adjacency_df.to_csv("adjacency_df-n"+str(n)+'-m'+str(m)+'-run'+str(elem+1)+"-risky"+str(int(risky_rate*100))+"-inf"+str(int(inf_rate*100))+"_"+str(name)+".csv",index=False)
            setV = range(1,n+1)
            impact_matrix = adjacency_df.copy(deep=True)
            impact_matrix.loc[:,setV]=impact_matrix.loc[:,setV].div(impact_matrix.loc[:,"Threshold"],axis=0)
            impact_matrix[impact_matrix[setV] >= 1] = 1
            Di_G = nx.DiGraph(G)
            for (u, v) in G.edges():
                Di_G.edges[u,v]['weight'] = impact_matrix.iat[v,u]
                Di_G.edges[v,u]['weight'] = impact_matrix.iat[u,v]
            node_features=pd.DataFrame(np.zeros((n,1)),columns=["IsInfected"],index=range(1,n+1))
            betweenness_cent=nx.betweenness_centrality(Di_G)
            degree_cent=nx.degree_centrality(Di_G)
            closeness_cent=nx.closeness_centrality(Di_G)
            eigenvector_cent=nx.eigenvector_centrality(Di_G)
            degree=Di_G.degree()
            node_features['IsInfected']=adjacency_df['IsInfected']
            node_features['IsRisky']=adjacency_df['IsRisky']
            node_features['map_for_dict']=betweenness_cent
            node_features['Betweenness']=node_features['map_for_dict'].map(betweenness_cent)
            node_features['Degree_Cent']=node_features['map_for_dict'].map(degree_cent)
            node_features['Closeness_cent']=node_features['map_for_dict'].map(closeness_cent)
            node_features['Eigenvector_cent']=node_features['map_for_dict'].map(eigenvector_cent)
            node_features['Degree']=node_features['map_for_dict'].map(degree)
            #sets
            setS = node_features.loc[(node_features.loc[:, "IsInfected"] < 1),"IsInfected"].index.tolist()
            setP = node_features.loc[(node_features.loc[:, "IsInfected"] > 0),"IsInfected"].index.tolist()
            setRisky= node_features.loc[(node_features.loc[:, "IsRisky"] > 0),"IsRisky"].index.tolist()
            node_features['Shortest_to_Infected']=9999
            node_features['Shortest_to_Risky']=0
            for i in setS:
                shortest_inf=np.array([9999])
                shortest_risky=np.array([9999])
                for j in setP:
                    if(nx.has_path(Di_G,source=i-1,target=j-1)):
                        shortest_inf=np.append(shortest_inf,nx.shortest_path_length(Di_G,source=i-1,target=j-1))
                node_features.at[i,'Shortest_to_Infected']=shortest_inf.min()
                for k in setRisky:
                    if(nx.has_path(Di_G,source=i-1,target=k-1)):
                        shortest_risky=np.append(shortest_risky,nx.shortest_path_length(Di_G,source=i-1,target=k-1))
                node_features.at[i,'Shortest_to_Risky']=shortest_risky.min()
            node_features['Total_Impact_Ratio']=impact_matrix[setV].sum(axis=1)
            os.chdir("C:\\Users\\Lenovo\\Desktop\\Experiments\\data\\node_features")
            #node_features.to_csv("node_features-n"+str(n)+'-m'+str(m)+'-run'+str(elem+1)+"-risky"+str(risky_rate*100)+"-inf"+str(inf_rate*100)+"_"+str(name)+".csv",index=False)
            node_features.to_csv("node_features-n"+str(n)+'-m'+str(m)+'-run'+str(elem+1)+"-risky"+str(int(risky_rate*100))+"-inf"+str(int(inf_rate*100))+"_"+str(name)+".csv",index=False)
            

            #sets
            setV = range(1,n+1)
            setS = adjacency_df.loc[(adjacency_df.loc[:, "IsInfected"] < 1),"IsInfected"].index.tolist()
            setP = adjacency_df.loc[(adjacency_df.loc[:, "IsInfected"] > 0),"IsInfected"].index.tolist()

            #parameters
            wij = [[adjacency_df.loc[i,j] for i in setV] for j in setV  ]
            ti = [adjacency_df.loc[i,"Threshold"] for i in setV]
            Xi = [adjacency_df.loc[i,"IsRisky"] for i in setV]
            #Blocking_Capacity = int((n) / 10) 
            Blocking_Capacity = int((n) / 10)*2

            start=time.time()
            #model
            Math_Model = gp.Model("Math_Model")

            #variables
            Yi = Math_Model.addVars(setV,vtype=GRB.BINARY,name="Yi") #Bunun yerine SetS'de tanımlı olabilir!!!!
            Ii = Math_Model.addVars(setV,vtype=GRB.BINARY, obj=Xi, name="Ii")
            Multi= Math_Model.addVars(setV,vtype=GRB.BINARY,name="Multi") #(1-Y[i])*(1-I[i])

            #constraints
            # Y[i] +I[i] <= 1   (1)
            for i in setV:
                Math_Model.addConstr((Yi[i]+ Ii[i]), sense=GRB.LESS_EQUAL, rhs=1, name="Bloklanan birey infected olamaz".format(i))

                # Mult[i] =  (1-Y[i])*(1-I[i])   (1)
            for i in setV:
                Math_Model.addConstr(Multi[i], sense=GRB.EQUAL, rhs=(1-Yi[i])*(1-Ii[i]), name="Multi constraint".format(i))#constraints
            # Y[i] +I[i] <= 1   (1)
            for i in setV:
                Math_Model.addConstr((Yi[i]+ Ii[i]), sense=GRB.LESS_EQUAL, rhs=1, name="Bloklanan birey infected olamaz".format(i))

            #(Σ(over j in V  w[i][j]*I[j]))*(1-Y[i])*(1-I[i]) <=  t[i]    for all i in S
            for i in setS:
                Math_Model.addConstr(lhs=gp.quicksum(wij[i-1][j-1]*Ii[j] for j in setV)*Multi[i] , sense=GRB.LESS_EQUAL, rhs=ti[i-1],name="Birinci:Tresholdu geçen birey hasta olur.".format(i))
            #(Σ(over j in V  w[i][j]*I[j]))*(1-Y[i]) >  t[i]*I[i]         for all i in S 
            for i in setS:
                Math_Model.addConstr(lhs=gp.quicksum(wij[i-1][j-1]*Ii[j] for j in setV)*(1-Yi[i]) , sense=GRB.GREATER_EQUAL, rhs=ti[i-1]*Ii[i],name="İkinci:Tresholdu geçen birey hasta olur.".format(i))

            #Σ(over i Y[i])<= c 
            Math_Model.addConstr(lhs=gp.quicksum(Yi[i] for i in setV) , sense=GRB.LESS_EQUAL, rhs=Blocking_Capacity,name="Bloklama kapasitesinden fazla bloklama yapılamaz.".format(i))

            #I[i]=1       for all i in V-S(P) 
            for i in setP:
                Math_Model.addConstr(Ii[i], sense=GRB.EQUAL, rhs=1, name="Başlangıçta enfekte olanların I[i] parametresi bire eşittir.".format(i))



            Math_Model.write("out.lp")
            # asd.computeIIS()
            # asd.write("model.ilp")

            Math_Model.ModelSense = GRB.MINIMIZE
            Math_Model.optimize()

            #for v in Math_Model.getVars():
                #if(v.x>= -0.0001):
                    #print('%s %g' % (v.varName, v.x))

            vars = Math_Model.getVars()

            adjacency_df['Blocked']=np.zeros((n,1))

            adjacency_df['Ending_Infected']=np.zeros((n,1))
            adjacency_df['Objective']=np.zeros((n,1))
            
            Objective_Value = Math_Model.ObjVal
            
            for i in range(1,n+1):
                adjacency_df.at[i,'Blocked']=vars[i-1].x
                adjacency_df.at[i,'Ending_Infected']=vars[i+n-1].x
                adjacency_df.at[i,'Objective']=Objective_Value
            
            
            #adjacency_df.loc[(adjacency_df.loc[:,'IsRisky'] == 1) & (adjacency_df.loc[:,'Ending_Infected'] == 1) ,['Threshold','IsInfected','IsRisky','Blocked','Ending_Infected']]
            os.chdir("C:\\Users\\Lenovo\\Desktop\\Experiments\\data\\DM_df")
            adjacency_df[['Blocked','Objective']].to_csv("Math_Model_NODE-n"+str(n)+'-m'+str(m)+'-run'+str(elem+1)+"-risky"+str(int(risky_rate*100))+"-inf"+str(int(inf_rate*100))+"_"+str(name)+".csv",index=False)
            end=time.time()


            
            


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2021-05-20
Using license file C:\Users\Lenovo\gurobi.lic
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 216 rows, 300 columns and 515 nonzeros
Model fingerprint: 0xd96dd4b8
Model has 270 quadratic constraints
Variable types: 0 continuous, 300 integer (300 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e-03, 1e+00]
  QLMatrix range   [1e-03, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
  QRHS range       [1e-03, 1e+00]
Presolve removed 119 rows and 135 columns
Presolve time: 0.01s
Presolved: 324 rows, 365 columns, 1448 nonzeros
Variable types: 0 continuous, 365 integer (365 binary)
Found heuristic solution: objective 46.0000000
Foun

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 211 rows, 300 columns and 510 nonzeros
Model fingerprint: 0xe8f121f5
Model has 280 quadratic constraints
Variable types: 0 continuous, 300 integer (300 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e-03, 1e+00]
  QLMatrix range   [1e-03, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
  QRHS range       [1e-03, 1e+00]
Presolve removed 87 rows and 110 columns
Presolve time: 0.01s
Presolved: 388 rows, 424 columns, 1754 nonzeros
Variable types: 0 continuous, 424 integer (424 binary)
Found heuristic solution: objective 46.0000000

Root relaxation: objective 2.051749e+01, 279 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It

Presolve removed 87 rows and 110 columns
Presolve time: 0.01s
Presolved: 388 rows, 424 columns, 1754 nonzeros
Variable types: 0 continuous, 424 integer (424 binary)
Found heuristic solution: objective 24.0000000

Root relaxation: objective 8.595841e+00, 295 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    8.59584    0   47   24.00000    8.59584  64.2%     -    0s
H    0     0                      12.0000000    8.59584  28.4%     -    0s
     0     0   10.00000    0    6   12.00000   10.00000  16.7%     -    0s
H    0     0                      11.0000000   10.00000  9.09%     -    0s
H    0     0                      10.0000000   10.00000  0.00%     -    0s
     0     0   10.00000    0    6   10.00000   10.00000  0.00%     -    0s

Cutting planes:
  Cover: 6
  Clique: 8
  MIR: 3
  StrongCG: 1
  RLT: 4

Explored 1 nodes (351 simplex iterations) 